In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import MatrixBasedConvolution as mbc

## Convolution parameters

In [2]:
inputs_shape_1d=(1,20)
inputs_shape_2d=(1,5,5,1)
ARGS={"kernel_size":3,
"stride":1,
"padding":'same',
"use_phi":True,
"activation":None,
"use_lambda_out":False,
"use_lambda_in":False}

## Convalution 1D and 2D outputs

In [3]:
print("1D Convolution building...")
con1d=mbc.matrix_conv_1d(**ARGS)
con1d.build(inputs_shape_1d)
print("2D Convolution building...")
con2d=mbc.matrix_conv_2d(**ARGS)
con2d.build(inputs_shape_2d)

1D Convolution building...
2D Convolution building...


In [4]:
print("inputs 1D generation...")
inputs_1d=tf.random.uniform(shape=inputs_shape_1d,minval=0,maxval=1)
print("inputs 2D generation...")
inputs_2d=tf.random.uniform(shape=inputs_shape_2d,minval=0,maxval=1)


inputs 1D generation...
inputs 2D generation...


In [5]:
ouputs_mbc_1d=con1d.conv_jit(inputs_1d).numpy()
print(ouputs_mbc_1d.shape)
ouputs_mbc_2d=con2d.conv_jit(inputs_2d).numpy()[0,:,:,0]
print(ouputs_mbc_2d.shape)

(1, 20)
(5, 5)


## Comparison between standard conv1d/conv2d and matrix based conv1d/conv2d

In [6]:
kernel=tf.reshape(con2d.kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size')))
kernel.shape

TensorShape([3, 3])

In [7]:
ouputs_classical=mbc.classical_convolution(img=inputs_2d[0,:,:,0],kernel=kernel.numpy())
ouputs_classical.shape


(5, 5)

In [8]:
ouputs_mbc_2d

array([[ 0.3059318 ,  0.38617957,  0.4018255 ,  0.3778597 ,  0.04954727],
       [ 0.32336083,  0.6207103 ,  0.22260246,  0.4291327 ,  0.26951125],
       [ 0.33101737,  0.83349425,  0.6769575 ,  0.16048278,  0.46369585],
       [ 0.30094528,  0.49383372,  0.58357507,  0.7501304 ,  0.37573946],
       [-0.00230748,  0.44241938,  0.32701463,  0.21546453,  0.21653849]],
      dtype=float32)

In [9]:
ouputs_classical

array([[ 0.3059318 ,  0.38617957,  0.40182546,  0.3778597 ,  0.04954726],
       [ 0.32336083,  0.6207103 ,  0.22260244,  0.4291327 ,  0.26951128],
       [ 0.33101737,  0.8334942 ,  0.6769575 ,  0.1604828 ,  0.46369585],
       [ 0.30094528,  0.49383372,  0.583575  ,  0.7501304 ,  0.37573946],
       [-0.00230748,  0.44241938,  0.32701457,  0.21546456,  0.21653849]],
      dtype=float32)

In [10]:
if np.allclose(ouputs_classical,ouputs_mbc_2d,atol=1e-16):
    print("The outputs are the same")


The outputs are the same


## Timer

In [11]:
%timeit ouputs_mbc_2d=con2d.conv_jit(inputs_2d).numpy()


127 μs ± 1.34 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%timeit mbc.classical_convolution(img=inputs_2d[0,:,:,0],kernel=kernel.numpy())

25.3 ms ± 284 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
conv_layer = tf.keras.layers.Conv2D(
filters=1,           
kernel_size=(ARGS.get('kernel_size'), ARGS.get('kernel_size')),     
strides=(ARGS.get('stride'), ARGS.get('stride')),  
activation=ARGS.get('activation'),      
padding=ARGS.get('padding'),        
use_bias=False          
)
conv_layer.build(input_shape=inputs_2d.shape)
conv_layer.set_weights([tf.reshape(kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size'),1,1))])


In [14]:
@tf.function(jit_compile=True)
def tf_conv2d(inputs_2d):
    return conv_layer(inputs_2d)


In [15]:
%timeit tf_conv2d(inputs_2d)

127 μs ± 3.2 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
